In [1]:
import pandas as pd
import json
from tqdm import tqdm
from collections import defaultdict
import pymongo
import re
import numpy as np
import pymongo
from rank_bm25 import BM25Okapi
import pickle

In [5]:
def post_process(data:dict,type='kakao'):
    # output : dict
    # key - doro addr
    output = defaultdict(list)
    c = 0
    # TODO
    if type=='kakao':
        for data_i in tqdm(data.values()):
            if data_i:
                for d in data_i:
                    name = d['name']
                    doro_address = d['address']
                    score = d['score']
                    reviews = []
                    for review in d['reviews']:
                        if review:
                            c+=1
                            review = review.split('\n')
                            date = review[5].strip() # TODO
                            text = review[6].strip() # TODO
                            reviews.append(dict(id=c, date=date, text=text))
                    output[doro_address].append(dict(name=name, score=score, jibun_address = '', reviews=reviews))
    elif type=='mango':
        for data_i in tqdm(data.values()):
            if data_i:
                for d in data_i:
                    if d:
                        name = d['score'].split('\n')[0] 
                        name = re.sub('[\(\)]','',name).strip()
                        try:
                            score = str(float(name[-3:])).strip()
                            name = name[:-3].strip()
                        except:
                            score = ''
                        detailed_info = d['detailed_info'].split('\n')
                        doro_addr=''
                        jibun_addr=''
                        for i in detailed_info:
                            if i.startswith('주소 '):
                                doro_addr = re.sub('주소 ','',i).strip()
                            elif i.startswith('지번 '):
                                jibun_addr = re.sub('지번 ','',i).strip()
                        reviews = []
                        for rev in d['review']:
                            if rev:
                                c+=1
                                rev = rev.split('\n')
                                # 사진 더보기 관련 해서 text 내용이 더 생기는 경우가 존재함
                                if '+' not in rev[-2].strip():
                                    date = rev[-2].strip()
                                    text = '\n'.join(rev[3:-2]).strip()
                                
                                elif '+' not in rev[-3].strip():
                                    date = rev[-3].strip()
                                    text = '\n'.join(rev[3:-3]).strip()
                                
                                reviews.append(dict(id=c, date=date, text=text))
                        if output[doro_addr]:
                            names = [n['name'] for n in output[doro_addr]]
                            if name in names:
                                continue
                            else:
                                output[doro_addr].append(dict(name=name, score=score, jibun_address = jibun_addr, reviews=reviews))
                        else:
                            output[doro_addr].append(dict(name=name, score=score, jibun_address = jibun_addr, reviews=reviews))
        
    result = {}
    c = 0
    for i,j in output.items(): 
        for k in j:
            k['doro_address']=i
            result[c]=k
            c+=1                
    return result

In [6]:
mango = json.load(open('../망고플레이트/강남구_output_mango_score.json'))

In [13]:
mango_post_process = post_process(mango,'mango')

100%|███████████████████████████████████████████████████████████████████████████| 1325/1325 [00:00<00:00, 13619.64it/s]


In [26]:
mango = json.load(open('../망고플레이트/강남구_output_mango_score.json'))

# BM25

In [14]:
tokenized_corpus = [(i['doro_address']+' '+i['name']).split(' ') for i in mango_post_process.values()]

In [15]:
bm25 = BM25Okapi(tokenized_corpus)

In [3]:
def retrieve(bm25, query):
    tokenized_query = query.split(' ')
    doc_scores = bm25.get_scores(tokenized_query)
    sorted_idx = np.argsort(-doc_scores)
    result = [i for i in sorted_idx]
    return result

In [72]:
pickle.dump(bm25, open('./bm25','wb'))

# mongodb

In [20]:
# connection
connection = pymongo.MongoClient() # basic port : 27017
# database instance화
database = connection.database # database에 database가 없어도 자동 생성함.
# document
company = database.affillation

In [24]:
for i,j in tqdm(mango_post_process.items(), desc='make_db'):
    j['id']=i
    post_id = company.insert_one(j)

make_db: 100%|█████████████████████████████████████████████████████████████████████| 444/444 [00:00<00:00, 1946.53it/s]


# query

In [2]:
bm25 = pickle.load(open('./bm25','rb'))
connection = pymongo.MongoClient() # basic port : 27017
database = connection.database # database에 database가 없어도 자동 생성함.
company = database.affillation

In [5]:
result = retrieve(bm25, '서울시 강남구 커스텀커피')

In [18]:
for i in company.find({'id':1}):
    print(i)

{'_id': ObjectId('63a555ce3ad7f16ce0b386f6'), 'name': '커스텀커피 개포점', 'score': '', 'jibun_address': '서울시 강남구 일원동 743 디에이치자이개포 단지내상가 808동 107호', 'reviews': [], 'doro_address': '서울특별시 강남구 영동대로 22 디에이치자이개포 단지내상가 808동 107호', 'id': 1}


In [22]:
answer = []
for i in result:
    for j in company.find({'id':int(i)}):
        answer.append(j)